In [1]:
import pandas as pd

In [2]:
from nltk.stem.porter import *
from nltk import bigrams, ngrams, word_tokenize
from nltk.corpus import stopwords

In [3]:
stemmer = PorterStemmer()

In [4]:
dat_narratives = pd.read_csv('../export/injury-narratives.csv')

In [5]:
tokenized_narratives = dat_narratives.NARRATIVE.fillna(
    ''
).str.lower().str.strip().apply(
    word_tokenize
).apply(
    lambda x: list(filter(
        lambda y: y not in set(stopwords.words('english')) and y.isalpha(), x
    ))
)

In [6]:
all_words = tokenized_narratives.apply(
    lambda x: pd.Series(x).unique()
).apply(pd.Series).stack()

In [7]:
# fail
# evidence
# reported
# arm
# supervise
# unsupervised
# play
# critical
# toy
# 48 hours

# [a for a in 
word_counts = all_words.value_counts().apply(pd.Series).rename(
    columns = {0: 'word count'}
)

In [11]:
word_counts.head(20)

word_counts.to_csv(
    '../csv/narrative-word-counts.csv'
)

In [9]:
stem_counts = all_words.apply(stemmer.stem).value_counts().apply(pd.Series).rename(
    columns = {0: 'word count'}
)

### Bigrams

In [15]:
tokenized_narratives.apply(
    bigrams
).apply(list).apply(
    pd.Series
).stack().apply(
    lambda x: ' '.join(x)
).value_counts().loc[
    lambda x: x >= 2
].append(
    # 6-grams
    tokenized_narratives.apply(
        lambda x: ngrams(x, 6)
    ).apply(list).apply(
        pd.Series
    ).stack().apply(
        lambda x: ' '.join(x)
    ).value_counts().loc[lambda x: x >= 3]
).append(
    # 4-grams
    tokenized_narratives.apply(
        lambda x: ngrams(x, 4)
    ).apply(list).apply(
        pd.Series
    ).stack().apply(
        lambda x: ' '.join(x)
    ).value_counts().loc[lambda x: x >= 3]
).sort_values(ascending=False).apply(
    pd.Series
).rename(
    columns = {0: 'count'}
).assign(
    is_jargon_phrase = lambda x: [
        any(
            jargon in index_phrase for jargon in ['based', 'obtained', 'standard', 'investigation found']
        ) for index_phrase in x.index
    ],
    phrase_length = lambda x: [len(a.split(' ')) for a in x.index]
).query('is_jargon_phrase == False').drop(
    'is_jargon_phrase', axis=1
).to_csv('../csv/narrative-phrase-counts.csv')